In [ ]:
# Unzip the data
def unzip_data(zip_dir, unzip_dir=None):
  """
  This model unzip a zip file. It takes in a zipfile and return and unzip file
  Args: 
    zip_dir: The path to the file to be unzipped
    unzip_dir: Directory to save the unzip file,
              None as a default value
  Return:
    Null
  """
  import zipfile
  zip_ref = zipfile.ZipFile(zip_dir)
  zip_ref.extractall(unzip_dir)
  zip_ref.close

In [ ]:
# Walk through the data
def walk_through_directory(directory):
  """
  This model walk through a directory and print the number of directories, files and the directories paths
  Args:
       directory: The path to the directory to walk through
  Return:
        Null
  """
  import os
  for dirpath, dirnames, filenames in os.walk(directory):
    print(f"There are {len(dirnames)} directories and {len(filenames)} files/images in '{dirpath}'.")

In [ ]:
# Plot random images from each class
def plot_random_images_from_each_class(train_dir):
  """
  Take in the training directory of a data and display a random image sample from a each class
  Args:
      train_dir(str): The directory path of the trainning data
  Return:
      Plot random images from each class of the data
  """
  import os
  import random
  import matplotlib.pyplot as plt 
  import matplotlib.image as mpimg 

  number_of_loop = 0

  plt.figure(figsize = (20, 15))
  num_classes = len(os.listdir(train_dir))

  # Get the subplot shape ready
  num_x_images = num_classes/2
  if (num_classes%2 == 1):
    num_x_images = num_x_images + 1

  for data_class in os.listdir(train_dir):
    number_of_loop +=1
    class_path = train_dir + "/" + data_class
    random_image_sample = random.choice(os.listdir(class_path))
    random_image_sample_path = class_path + "/" + random_image_sample

    # Load and plot the image
    image = mpimg.imread(random_image_sample_path)
    plt.subplot(num_x_images, num_x_images, number_of_loop)
    plt.imshow(image)
    plt.title(data_class)
    plt.axis(False)

In [ ]:
# Load the data into train_data and test_data
def load_data(train_dir, test_dir, img_size=(224, 224), class_mode = "categorical", batch_size=32):
  """
  """
  import tensorflow as tf

  print("Loading training data")
  train_data = tf.keras.utils.image_dataset_from_directory(directory=train_dir,
                                                           label_mode= class_mode,
                                                           image_size= img_size,
                                                           batch_size = batch_size)
                                                           
  print("Loading test data")
  test_data = tf.keras.utils.image_dataset_from_directory(directory=test_dir,
                                                          label_mode=class_mode,
                                                          image_size=img_size,
                                                          batch_size=batch_size)
  
  return train_data, test_data

In [ ]:
def plot_augumented_image(train_dir):
  """
  """
  import os
  import random
  import matplotlib.pyplot as plt
  import matplotlib.image as mpimg
  random_class = random.choice(os.listdir(train_dir))
  random_class_path = train_dir + "/" + random_class
  random_image_sample = random.choice(os.listdir(random_class_path))
  random_image_sample_path = random_class_path + "/" + random_image_sample

  # Read in the image
  image = mpimg.imread(random_image_sample_path)
  aug_image = data_augumentation(image)
  plt.subplot(1,2,1)
  plt.imshow(image)
  plt.title(random_class)
  plt.subplot(1,2,2)
  plt.imshow(aug_image/255.)
  plt.title(random_class)
  plt.axis(False);



In [ ]:
# let's create function to plt our loss curve
def plot_loss_curve(history):
  """
  Return seperate loss curves for loss and accuracy

  Args: 
    history: TensorFlow History object.

  Returns:
    plot of treaining/validation loss and accuracy metrics
  """
  import pandas as pd
  import matplotlib.pyplot as plt
  history_dataframe = pd.DataFrame(history.history)
  train_loss = history_dataframe["loss"]
  val_loss = history_dataframe["val_loss"]
  train_accuracy = history_dataframe["accuracy"]
  val_accuracy = history_dataframe["val_accuracy"]
  epochs = range(len(train_loss))

  plt.figure();
  plt.plot(epochs, train_loss)
  plt.plot(epochs, val_loss)
  plt.title("loss")
  plt.legend(["training_loss", "val_loss"])
  plt.xlabel("epochs")

  plt.figure();
  plt.plot(epochs, train_accuracy)
  plt.plot(epochs, val_accuracy)
  plt.title('Accuracy')
  plt.legend(["training_accuracy", "Val_accuracy"])
  plt.xlabel("epochs")


# New Section

In [ ]:
# Create tensorboard callback
def create_tensorboard_callback(directory_name, experiment_name):
  """
  """
  import tensorflow as tf
  import datetime
  log_dir = directory_name + "/" + experiment_name + datetime.datetime.now().strftime("%y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard callback in '{log_dir}'.")
  return tensorboard_callback


In [ ]:
def create_checkpoint_callback(checkpoint_path):
  import tensorflow as tf
  checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True,
                                                         save_best_only=False,
                                                         save_freq="epoch",
                                                         verbose = 1)
  return checkpoint_callback

In [ ]:
def fine_tune(base_model, num_of_ft_layers):
  num_of_ft_layers = int(num_of_ft_layers) 
  base_model.trainable = True
  for layers in base_model.layers[:-num_of_ft_layers]:
    layers.trainable = False

In [2]:
def get_percent_images(target_dir, new_dir, sample_amount=0.1, random_state=42):
    """
    Get sample_amount percentage of random images from target_dir and copy them to new_dir.
    
    Preserves subdirectory file names.
    
    E.g. target_dir=pizza_steak/train/steak/all_files 
                -> new_dir_name/train/steak/X_percent_of_all_files
                
    Parameters
    --------
    target_dir (str) - file path of directory you want to extract images from
    new_dir (str) - new directory path you want to copy original images to
    sample_amount (float), default 0.1 - percentage of images to copy (e.g. 0.1 = 10%)
    random_state (int), default 42 - random seed value 
    """
    # Set random seed for reproducibility
    import random
    import shutil
    import os
    from tqdm import tqdm
    
    random.seed(random_state)
    
    # Get a list of dictionaries of image files in target_dir
    # e.g. [{"class_name":["2348348.jpg", "2829119.jpg"]}]
    images = [{dir_name: os.listdir(target_dir + dir_name)} for dir_name in os.listdir(target_dir)]

    for i in images:
        for k, v in i.items():
            # How many images to sample?
            sample_number = round(int(len(v)*sample_amount))
            print(f"There are {len(v)} total images in '{target_dir+k}' so we're going to copy {sample_number} to the new directory.")
            print(f"Getting {sample_number} random images for {k}...")
            random_images = random.sample(v, sample_number)

            # Make new dir for each key
            new_target_dir = new_dir + k
            print(f"Making dir: {new_target_dir}")
            os.makedirs(new_target_dir, exist_ok=True)

            # Keep track of images moved
            images_moved = []

            # Create file paths for original images and new file target
            print(f"Copying images from: {target_dir}\n\t\t to: {new_target_dir}/\n")
            for file_name in tqdm(random_images):
                og_path = target_dir + k + "/" + file_name
                new_path = new_target_dir + "/" + file_name

                # Copy images from OG path to new path
                shutil.copy2(og_path, new_path)
                images_moved.append(new_path)

            # Make sure number of images moved is correct
            assert len(os.listdir(new_target_dir)) == sample_number
            assert len(images_moved) == sample_number